In [8]:
"""Startup"""
%matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


pathToDataCollectorDatabase = 'C:\\Users\\mcall\\Desktop\\test\\sca-ml\\Jupyter Notebooks\\Testing\\PythonTest\\'

In [9]:
def searchFiles(path, fileformats):
    filesToDo = []
    for root, dirs, files in os.walk(path):
        for file in files:
            for fileformat in fileformats:
                if file.endswith(fileformat):
                    #code to generate a list of paths of files to generate logs for
                    filesToDo.append(os.path.join(root, file).removeprefix(path))
    return filesToDo

def searchFileName(path, fileName):
    filesToDo = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file == fileName:
                #code to generate a list of paths of files to generate logs for
                filesToDo.append(os.path.join(root, file).removeprefix(path))
    return filesToDo

In [10]:

filesToIndex = searchFileName(pathToDataCollectorDatabase,"gitinfo.txt")
#Create a dataframe that contains the data of the gitinfo.txt files
#example of the gitinfo.txt file    {"id": 3452233, "name": "hjlebbink/asm-dude", "isFork": false, "commits": 939, "branches": 8, "defaultBranch": "master", "releases": 39, "contributors": 10, "license": "MIT License", "watchers": 762, "stargazers": 3906, "forks": 76, "size": 82262, "createdAt": "2016-02-19 03:43:23", "pushedAt": "2022-01-06 01:33:34", "updatedAt": "2022-01-08 08:02:49", "homepage": "", "mainLanguage": "Python", "totalIssues": 118, "openIssues": 32, "totalPullRequests": 13, "openPullRequests": 1, "lastCommit": "2021-12-07 08:47:07", "lastCommitSHA": "1e1e3dc2364e8ed549ba404868b6473ba435b25c", "hasWiki": true, "isArchived": false, "languages": {"Python": 4274689, "C#": 2554588, "C": 1311140, "C++": 644077, "Assembly": 164163, "Shell": 2764, "Go": 1779, "Batchfile": 410}, "labels": ["bug", "duplicate", "enhancement", "help wanted", "invalid", "question", "wontfix"]}
df = pd.DataFrame(columns=['id', 'name', 'isFork', 'commits', 'branches', 'defaultBranch', 'releases', 'contributors', 'license', 'watchers', 'stargazers', 'forks', 'size', 'createdAt', 'pushedAt', 'updatedAt', 'homepage', 'mainLanguage', 'totalIssues', 'openIssues', 'totalPullRequests', 'openPullRequests', 'lastCommit', 'lastCommitSHA', 'hasWiki', 'isArchived', 'languages', 'labels' , 'path'])
for file in filesToIndex:
    dftemp = pd.read_json (pathToDataCollectorDatabase+file, lines=True)
    #Proterly format the booleans
    dftemp['isFork'] = dftemp['isFork'].astype('bool')
    dftemp['hasWiki'] = dftemp['hasWiki'].astype('bool')
    dftemp['isArchived'] = dftemp['isArchived'].astype('bool')
    #Properly format the dates
    dftemp['createdAt'] = pd.to_datetime(dftemp['createdAt'])
    dftemp['pushedAt'] = pd.to_datetime(dftemp['pushedAt'])
    dftemp['updatedAt'] = pd.to_datetime(dftemp['updatedAt'])
    dftemp['lastCommit'] = pd.to_datetime(dftemp['lastCommit'])
    #Properly format the languages
    dftemp['languages'] = dftemp['languages'].astype('str')
    #Properly format the labels
    dftemp['labels'] = dftemp['labels'].astype('str')
    #Add the path to the file
    dftemp['path'] = file.removesuffix('gitinfo.txt')
    #Append the data to the dataframe
    df = pd.concat([df, dftemp], ignore_index=True)
df.head()


C:\Users\mcall\AppData\Local\Temp\ipykernel_16772\3565412589.py:23: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, dftemp], ignore_index=True)
C:\Users\mcall\AppData\Local\Temp\ipykernel_16772\3565412589.py:23: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, dftemp], ignore_index=True)
C:\Users\mcall\AppData\Local\Temp\ipykernel_16772\3565412589.py:23: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, dftemp], ignore_index=True)
C:\Users\mcall\AppData\Local\Temp\ipykernel_16772\3565412589.py:23: FutureWarning: In a future version, object-dtype columns wi

,id,name,isFork,commits,branches,defaultBranch,releases,contributors,license,watchers,...,openIssues,totalPullRequests,openPullRequests,lastCommit,lastCommitSHA,hasWiki,isArchived,languages,labels,path
0,3452233,hjlebbink/asm-dude,False,939,8,master,39,10,MIT License,762,...,32,13,1,2021-12-07 08:47:07,1e1e3dc2364e8ed549ba404868b6473ba435b25c,True,False,"{'Python': 4274689, 'C#': 2554588, 'C': 131114...","['bug', 'duplicate', 'enhancement', 'help want...",asm-dude\
1,3275794,googlecloudplatform/community,False,1430,9,master,0,346,Other,198,...,31,1500,33,2022-09-07 10:02:08,e46b6ea2c47389c2ca19e033855f4e206f28fc51,False,False,"{'Java': 358060, 'Python': 292134, 'Jupyter No...","['DO NOT MERGE', 'automerge', 'automerge: exac...",community\
2,3277687,feast-dev/feast,False,2636,30,master,69,135,Apache License 2.0,68,...,129,2081,18,2022-07-16 04:29:09,476fccd322c684f44fd17c942341e4d0082c8526,False,False,"{'Python': 864999, 'Java': 260607, 'Shell': 40...","['Community Contribution Needed', 'approved', ...",feast\
3,2444449,coursera/programming-assignments-demo,False,124,21,master,0,10,NaN,31,...,2,34,1,2021-11-30 07:42:40,7ccb36f4550e5e0b44b1774d8bd3ea4ec1b5a2a5,True,False,"{'Python': 25480, 'Java': 19415, 'Dockerfile':...","['bug', 'duplicate', 'enhancement', 'help want...",programming-assignments-demo\
4,3275944,returntocorp/semgrep-rules,False,2234,171,develop,0,140,Other,24,...,22,1870,12,2022-09-02 05:45:18,7ae34f1ef07022d3b318f43aa3cafc646af944be,False,False,"{'Java': 383936, 'Python': 353931, 'JavaScript...","['DTL', 'Hacktoberfest', 'SQLi', 'SSRF', 'XSS'...",semgrep-rules\


In [11]:
#Create a data

In [12]:
#EXPERIMENT 1: Grade Generation Based off of the number of commits
#Formula: (commit/totalCommits)
#This is a very simple formula that will be used to generate a grade for the number of commits as the first dataset used to train the model
#read the file path from the dataframe
filesToIndex = df['path'].tolist()
#Create an array of strings of folders which contain a 0.txt file
foldersWithCommits = []
for file in filesToIndex:
    for filelvl2 in searchFileName(pathToDataCollectorDatabase+file, "0.txt"):
        foldersWithCommits.append(file+filelvl2.removesuffix('0.txt'))
print (foldersWithCommits)

#get the total number of commits for each folder
totalCommits = []
for folder in foldersWithCommits:
    totalCommits.append(len(searchFiles(pathToDataCollectorDatabase+folder, ['.txt'])))
    print(pathToDataCollectorDatabase+folder)
print (totalCommits)

dataSet1 = pd.DataFrame(columns=['data', 'grade'])
for i in foldersWithCommits:
    for j in range(len(searchFiles(pathToDataCollectorDatabase+i, ['.txt']))):
        #open 'j'.txt and read it
        data = open(pathToDataCollectorDatabase+i+str(j)+'.txt', 'r', encoding="utf8")
        #add the data to the dataframe
        dataSet1temp = pd.DataFrame({'data': data, 'grade': j/len(searchFiles(pathToDataCollectorDatabase+i, ['.txt']))})
        dataSet1 = pd.concat([dataSet1, dataSet1temp], ignore_index=True)
        #close the file
        data.close()
dataSet1.head()

['asm-dude\\parsed\\main\\example\\', 'asm-dude\\parsed\\main\\get-pip\\', 'asm-dude\\parsed\\main\\main\\', 'asm-dude\\parsed\\main\\parser\\', 'asm-dude\\parsed\\main\\pile\\', 'asm-dude\\parsed\\main\\writer\\', 'asm-dude\\parsed\\main\\z3\\', 'asm-dude\\parsed\\main\\z3consts\\', 'asm-dude\\parsed\\main\\z3core\\', 'asm-dude\\parsed\\main\\z3num\\', 'asm-dude\\parsed\\main\\z3poly\\', 'asm-dude\\parsed\\main\\z3printer\\', 'asm-dude\\parsed\\main\\z3rcf\\', 'asm-dude\\parsed\\main\\z3types\\', 'asm-dude\\parsed\\main\\z3util\\', 'asm-dude\\parsed\\main\\__init__\\', 'community\\parsed\\main\\activate_agent\\', 'community\\parsed\\main\\app\\', 'community\\parsed\\main\\ar-subs\\', 'community\\parsed\\main\\ardu-pi-serial\\', 'community\\parsed\\main\\avdemo\\', 'community\\parsed\\main\\batch\\', 'community\\parsed\\main\\batch_test\\', 'community\\parsed\\main\\batch_tokenize_and_detokenize\\', 'community\\parsed\\main\\beam-solarwind\\', 'community\\parsed\\main\\bom-loader\\', '

,data,grade
0,"# Copyright (c) Microsoft Corporation 2015, 20...",0.0
1,\n,0.0
2,# The Z3 Python API requires libz3.dll/.so/.dy...,0.0
3,# PATH/LD_LIBRARY_PATH/DYLD_LIBRARY_PATH\n,0.0
4,# environment variable and the PYTHONPATH envi...,0.0


In [13]:
#Dataset Structure (Data, Grade Value)
#Grade Value: 0 = Bad, 1 = Good (Float)
#Data: (String)

#Create a dataset 






In [14]:
""Monte Carlo Simulation""


SyntaxError: invalid syntax (1721608727.py, line 1)

In [15]:
"Basic Nural Network"
#Create a neural network to corelate the strings and the grade values
import torch
import torch.nn as nn
import torch.nn.functional as F
from tensorflow.keras.preprocessing.text import Tokenizer

class Net(nn.Module):
        def __init__(self):
            super().__init__()
            self.fc1 = nn.Linear(1, 1)
    
        def forward(self, x):
            x = F.relu(self.fc1(x))
            return x

net = Net()
print(net)

input = dataSet1['data'].tolist()
target = dataSet1['grade'].tolist()


tokenizer = Tokenizer()
tokenizer.fit_on_texts(input)

Net(
  (fc1): Linear(in_features=1, out_features=1, bias=True)
)


ValueError: expected sequence of length 6 at dim 1 (got 0)